<a href="https://colab.research.google.com/github/wizardoftrap/llm-notebooks/blob/main/functionGemma_fineTune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install unsloth
!pip install transformers==4.57.1
!pip install --no-deps trl==0.22.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 3.6 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.1/376.1 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.7/290.7 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/

In [ ]:
import torch
from unsloth import FastLanguageModel

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
gemma_model = "google/functiongemma-270m-it"
base_model, tokenizer = FastLanguageModel.from_pretrained(
    gemma_model,
    device_map="auto")

==((====))==  Unsloth 2025.12.9: Fast Gemma3 patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.1+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.5.1
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.
Unsloth: Gemma3 does not support SDPA - switching to fast eager.


model.safetensors:   0%|          | 0.00/393M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/205 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/63.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/714 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [ ]:
import json
from random import randint
from datasets import load_dataset
from transformers import AutoTokenizer
from huggingface_hub import hf_hub_download

data_file = hf_hub_download(repo_id="google/mobile-actions", filename="dataset.jsonl", repo_type="dataset")
dataset = load_dataset("text", data_files=data_file, encoding="utf-8")["train"].shuffle()

print(f"\n\033[1mHere's an example from your dataset:\033[0m \n{json.dumps(json.loads(dataset[randint(0, len(dataset) - 1)]['text']), indent=2)}")

dataset.jsonl:   0%|          | 0.00/25.7M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]


Here's an example from your dataset: 
{
  "metadata": "train",
  "tools": [
    {
      "function": {
        "name": "create_contact",
        "description": "Creates a contact in the phone's contact list.",
        "parameters": {
          "type": "OBJECT",
          "properties": {
            "last_name": {
              "type": "STRING",
              "description": "The last name of the contact."
            },
            "first_name": {
              "type": "STRING",
              "description": "The first name of the contact."
            },
            "email": {
              "type": "STRING",
              "description": "The email address of the contact."
            },
            "phone_number": {
              "type": "STRING",
              "description": "The phone number of the contact."
            }
          },
          "required": [
            "first_name",
            "last_name"
          ]
        }
      }
    },
    {
      "function": {
        "name":

In [ ]:
import json

def apply_format(sample):
  template_iputs = json.loads(sample['text'])

  prompt_and_completion = tokenizer.apply_chat_template(
    template_iputs['messages'],
    tools=template_iputs['tools'],
    tokenize=False,
    # add_generation_prompt is False since we don't need model output after all
    # messages.
    add_generation_prompt=False)

  prompt = tokenizer.apply_chat_template(
    template_iputs['messages'][:-1],
    tools=template_iputs['tools'],
    tokenize=False,
    # add_generation_prompt is True since we would like to include
    # "<start_of_turn>model" in the prompt, if needed.
    add_generation_prompt=True)

  completion = prompt_and_completion[len(prompt):]

  return {
     "prompt": prompt,
     "completion": completion,
     "split": template_iputs["metadata"],
  }

processed_dataset = dataset.map(apply_format)

Map:   0%|          | 0/9654 [00:00<?, ? examples/s]

In [ ]:
#@title Review the processed dataset

print("\033[1mHere's an example from the formatted dataset:\033[0m")
print(json.dumps(processed_dataset[randint(0, len(processed_dataset) - 1)], indent=2))

longest_example = max(processed_dataset, key=lambda example: len(example['prompt'] + example['completion']))
longest_example_token_count = len(tokenizer.tokenize(longest_example['prompt'] + longest_example['completion']))

print(f"\n\033[1mThe longest example length is {len(longest_example['prompt'] + longest_example['completion'])} with {longest_example_token_count} tokens. We need to set the max_length larger than the token count in SFTConfig below.\033[0m")
print(json.dumps(longest_example, indent=2))

max_token_count = longest_example_token_count + 100
print(f"\n\033[1mUsing max_token_count of {max_token_count} (= {longest_example_token_count} + 100) for training below.\033[0m")

Here's an example from the formatted dataset:
{
  "text": "{\"metadata\": \"train\", \"tools\": [{\"function\": {\"name\": \"create_contact\", \"description\": \"Creates a contact in the phone's contact list.\", \"parameters\": {\"type\": \"OBJECT\", \"properties\": {\"first_name\": {\"type\": \"STRING\", \"description\": \"The first name of the contact.\"}, \"last_name\": {\"type\": \"STRING\", \"description\": \"The last name of the contact.\"}, \"phone_number\": {\"type\": \"STRING\", \"description\": \"The phone number of the contact.\"}, \"email\": {\"type\": \"STRING\", \"description\": \"The email address of the contact.\"}}, \"required\": [\"first_name\", \"last_name\"]}}}, {\"function\": {\"name\": \"turn_off_flashlight\", \"description\": \"Turns the flashlight off.\", \"parameters\": {\"type\": \"OBJECT\", \"properties\": {}}}}, {\"function\": {\"name\": \"turn_on_flashlight\", \"description\": \"Turns the flashlight on.\", \"parameters\": {\"type\": \"OBJECT\", \"properties

In [ ]:
#@title Prepare train and eval dataset.

train_dataset = processed_dataset.filter(lambda example: example['split'] == 'train')
eval_dataset = processed_dataset.filter(lambda example: example['split'] == 'eval')

Filter:   0%|          | 0/9654 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9654 [00:00<?, ? examples/s]

In [ ]:
from huggingface_hub import login
login()

In [ ]:
base_model=FastLanguageModel.get_peft_model(
    base_model,
    r=64,
    lora_alpha=64,
    lora_dropout=0.0,
    finetune_language_layers=True,
    finetune_attention_modules=True,
    finetune_mlp_modules=True,
    random_state=3407)

Unsloth: Making `model.base_model.model.model` require gradients


In [ ]:
from trl import SFTTrainer, SFTConfig
output_dir="output"
FastLanguageModel.for_training(base_model) # Enable for training!
args = SFTConfig(
    output_dir=output_dir,
    num_train_epochs=2,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    logging_strategy="steps",                         # Log every steps
    eval_strategy="steps",
    eval_steps=50,
    logging_steps=50,
    save_strategy="epoch",
    learning_rate=1e-5,                               # Learning rate,
    lr_scheduler_type="cosine",
    max_length=max_token_count,
    gradient_checkpointing=True,
    packing=False,
    optim="adamw_torch_fused",
    completion_only_loss=True,
    report_to="none"                                  # No reporting.
)
trainer = SFTTrainer(
    model = base_model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    args = args,
)

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/8693 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/961 [00:00<?, ? examples/s]

🦥 Unsloth: Padding-free auto-enabled, enabling faster training.


In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 8,693 | Num Epochs = 2 | Total steps = 544
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 8 x 1) = 32
 "-____-"     Trainable parameters = 15,187,968 of 283,286,144 (5.36% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
50,1.520500,3.436320
100,1.359300,4.777485
150,2.169200,4.665601
200,1.267200,4.751897
250,0.861300,4.866707
300,0.642300,5.099219
350,0.563300,5.275068
400,0.525600,5.408698
450,0.509600,5.472287
500,0.504900,5.494019


Unsloth: Not an error, but Gemma3ForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


In [ ]:
base_model.push_to_hub_merged(
        "wizardoftrap/functiongemma-270m-it-mobile-actions", tokenizer,
        token = "hf_H****"
    )

config.json: 0.00B [00:00, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...e-actions/tokenizer.model: 100%|##########| 4.69MB / 4.69MB            

  ...le-actions/tokenizer.json:  25%|##4       | 8.28MB / 33.4MB            

...

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...actions/model.safetensors:   5%|4         | 25.1MB /  536MB            

Unsloth: Merging weights into 16bit: 100%|██████████| 1/1 [00:21<00:00, 21.53s/it]


Unsloth: Merge process complete. Saved to `/content/wizardoftrap/functiongemma-270m-it-mobile-actions`
